# IMPORTS

In [33]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
from parse import compile
import matplotlib.pyplot as plt
import numpy as np
import heapq

# Part 1

In [61]:
edge_encoded = compile("{} to {} = {:d}")

with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    # step 1: count number of distinct nodes
    seen = set()
    index = {}
    for line in data:
        u, v, w = edge_encoded.parse(line).fixed
        if u not in seen:
            index[u] = len(index)
            seen.add(u)
        if v not in seen:
            index[v] = len(index)
            seen.add(v)
    n = len(seen)
    # step 2: read in the all pairs shortest distance
    dist = [[0] * n for _ in range(n)]
    for line in data:
        u, v, w = edge_encoded.parse(line).fixed
        dist[index[u]][index[v]] = w
        dist[index[v]][index[u]] = w

    # step 3: run dynamic programming
    res = math.inf
    # start a trip from each node
    for src in range(n):
        dp = [[math.inf] * n for _ in range(1 << n)]
        dp[1 << src][src] = 0
        for mask in range(1 << src, 1 << n):
            for u in range(n):
                if dp[mask][u] == math.inf: continue
                for v in range(n):
                    if (mask >> v) & 1: continue # already visited
                    nmask = mask | (1 << v)
                    dp[nmask][v] = min(dp[nmask][v], dp[mask][u] + dist[u][v])
        # for all routes that visit every node and the last visited node is i
        for i in range(n):
            res = min(res, dp[(1 << n) - 1][i])
    print(res)
    


141


# Part 2

In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))